In [334]:
#update libs
!pip install --upgrade pandas
!pip install --upgrade scikit-learn
!pip install --upgrade inflect

Requirement already up-to-date: pandas in c:\anaconda\lib\site-packages (0.23.4)
Requirement already up-to-date: scikit-learn in c:\anaconda\lib\site-packages (0.20.2)


In [335]:
#import common libs
import numpy as np
import pandas as pd
import scipy.stats as st
from matplotlib import pyplot as plt
import seaborn as sns

#from scipy import interp
sns.set(font_scale=1.3)

In [336]:
#import special libs
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, Imputer,OrdinalEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import PowerTransformer,QuantileTransformer
from sklearn.preprocessing import label_binarize,MultiLabelBinarizer
from sklearn.pipeline import make_pipeline,Pipeline

from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

from itertools import cycle

from sklearn.multiclass import OneVsRestClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC,LinearSVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split 
from sklearn.model_selection import KFold,RandomizedSearchCV, cross_val_score

from sklearn.metrics import confusion_matrix, recall_score, brier_score_loss, roc_curve, auc
from sklearn.metrics import f1_score, make_scorer, roc_auc_score, precision_score
from sklearn.metrics import classification_report

from sklearn.decomposition import PCA, TruncatedSVD,IncrementalPCA

import glob

from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.calibration import CalibratedClassifierCV, calibration_curve


%matplotlib inline

In [465]:
import nltk
nltk.download('punkt')
nltk.download("stopwords")
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer # one more stemmer? or lemmatisation
from nltk.stem import WordNetLemmatizer,LancasterStemmer

import re
import string
import inflect

seed = 321

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chief\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chief\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\chief\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


Загрузим датасет с классифицируемыми текстами.

In [102]:
from sklearn.datasets import fetch_20newsgroups

Итак, задача данной работы:



#### EDA.

Посмотрим на данные, для этого:
- скачаем датасет целиком
- посмотрим на категории текстов
- распределение документов по категориям
- посмотрим на примеры данных

In [513]:

unique, counts = np.unique(newsgroups_all.target, return_counts=True)
unique, counts
#pprint(newsgroups_all.data[1])

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19]),
 array([799, 973, 985, 982, 963, 988, 975, 990, 996, 994, 999, 991, 984,
        990, 987, 997, 910, 940, 775, 628], dtype=int64))

In [521]:
len(newsgroups_all.target[newsgroups_all.target == 0])

799

In [522]:
# print the histogramm with distribution of texts by categorieas and topics

In [391]:
# здесь shuffle = true, т.к. мы собираемся использовать SGDClassifier; Naive Bayes - они базируются на принципе iid данных 
#и чтобы не получить смещенную оценку
#newsgroups_test = fetch_20newsgroups(subset='test', random_state=seed, shuffle=True, remove = ('headers', 'footers', 'quotes'))
#newsgroups_train = fetch_20newsgroups(subset='train', random_state=seed, shuffle=True, remove = ('headers', 'footers', 'quotes'))
newsgroups_all = fetch_20newsgroups(subset='all', random_state=seed, shuffle=True, remove = ('headers', 'footers', 'quotes'))

from pprint import pprint
pprint(list(newsgroups_all.target_names))

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


In [256]:
print("%d documents" % len(newsgroups_all.data))
print("%d categories" % len(newsgroups_all.target_names))

18846 documents
20 categories


Посмотрим на образец данных.

In [450]:
pprint(newsgroups_all.data[3])

("You can't call time when there's a play in progress.\n"
 '\n'
 'Ryan Robbins\n'
 'Penobscot Hall\n'
 'University of Maine')


In [332]:
newsgroups_all.target_names[newsgroups_all.target[1]]

'sci.space'

#### Data preprocessing.

Предобработаем данные, для этого:
- приведем все слова к нижнему регистру+.
- удалим знаки препинания+.
- заменим представление чисел в виде цифр на текстовые аналоги+.
- отсечем стоп слова (уберем символы и комбинации, которые не составляют смысловой нагрузки)+.
- проведем лемматизацию (приведение слова к начальной форме с помощью словаря)+.
- выделим n-граммы (последовательности из нескольких слов, рассматриваемых как одно целое).

In [400]:
def remove_punctuation(text):
    new_text = []
    for word in text:
        new_word = re.sub('['+text_punctuation+']', '', word)
        if new_word != '':
            new_text.append(new_word)
    
    return new_text

In [401]:
def digits_to_text(text):
    p = inflect.engine()
    new_text = []
    for word in text:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_text.append(new_word)
        else:
            new_text.append(word)
    return new_text

In [461]:
def lemmatize_stemm_text(text):
    new_text_lemma = []
    new_text_stemm = []
    
    lemmatizer = WordNetLemmatizer()
    stemmer = LancasterStemmer()
    
    for word in text:
        lemma = lemmatizer.lemmatize(word, pos='v')
        new_text_lemma.append(lemma)
        
        stemm = stemmer.stem(word)
        new_text_stemm.append(stemm)
        
    return new_text_lemma, new_text_stemm
        

In [475]:
StopWords = stopwords.words('english')
newStopWords = ['jfc','jb', 'cant'] # по ходу работы периодически добавляем стоп-слова
StopWords.extend(newStopWords)

# getting source from string.punctuation, but removed right slash / and ^
text_punctuation = '!"#$%&\'()*+,-.:;<=>?@[\\]_`{|}~'

def tokenize(text):
    min_length = 3
    
    #remove punctuation
    text = re.sub('['+text_punctuation+']', '',text)
    text = re.sub('\n', ' ',text)
    
    
    # tokenize text
    words = word_tokenize(text,language='english')
    
    #words to lowercase
    words = map(lambda word: word.lower(), words)
    
    #replace digits with words
    words = digits_to_text(words)
        
    # remove stopwords
    # обогатить stopwords? возможно стоит взять слова с tf-idf
    words = [word for word in words
                  if word not in StopWords]
    
    #lemmatize or stemm words 
    words,_ = lemmatize_stemm_text(words)
    
    
    #tokens = (list(map(lambda token: PorterStemmer().stem(token),
    #              words)));
    #p = re.compile('[a-zA-Z]+');
    #filtered_tokens = list(filter(lambda token:
    #              p.match(token) and len(token)>=min_length,tokens))
    return words#filtered_tokens

In [476]:
pprint(newsgroups_all.data[1])

('\n'
 'jfc> If gamma ray bursters are extragalactic, would absorption from the\n'
 'jfc> galaxy be expected?  How transparent is the galactic core to gamma\n'
 'jfc> rays?\n'
 '\n'
 'and later...\n'
 '\n'
 'JB> So, if the 1/r^2 law is incorrect (assume\n'
 'JB> some unknown material [dark matter??] inhibits Gamma Ray propagation),\n'
 'JB> could it be possible that we are actually seeing much less energetic\n'
 'JB> events happening much closer to us?  The even distribution could\n'
 'JB> be caused by the characteristic propagation distance of gamma rays \n'
 'JB> being shorter then 1/2 the thickness of the disk of the galaxy.\n'
 '\n'
 '\n'
 ' 0.\n'
 '\n'
 ' Well, maybe not zero, but very little.  At the typical energies for \n'
 ' gamma rays, the Galaxy is effectively transparent. \n'
 '\n'
 ' Hans Bloemen had a review article in Ann. Rev. Astr. Astrophys. a few \n'
 ' years back in which he discusses this in more depth.')


In [477]:
% time
test = tokenize(newsgroups_all.data[1])

Wall time: 0 ns


In [478]:
pprint(test)

['gamma',
 'ray',
 'bursters',
 'extragalactic',
 'would',
 'absorption',
 'galaxy',
 'expect',
 'transparent',
 'galactic',
 'core',
 'gamma',
 'ray',
 'later',
 '1/r^2',
 'law',
 'incorrect',
 'assume',
 'unknown',
 'material',
 'dark',
 'matter',
 'inhibit',
 'gamma',
 'ray',
 'propagation',
 'could',
 'possible',
 'actually',
 'see',
 'much',
 'less',
 'energetic',
 'events',
 'happen',
 'much',
 'closer',
 'us',
 'even',
 'distribution',
 'could',
 'cause',
 'characteristic',
 'propagation',
 'distance',
 'gamma',
 'ray',
 'shorter',
 '1/2',
 'thickness',
 'disk',
 'galaxy',
 'zero',
 'well',
 'maybe',
 'zero',
 'little',
 'typical',
 'energies',
 'gamma',
 'ray',
 'galaxy',
 'effectively',
 'transparent',
 'hans',
 'bloemen',
 'review',
 'article',
 'ann',
 'rev',
 'astr',
 'astrophys',
 'years',
 'back',
 'discuss',
 'depth']


In [254]:
#new_sentence = re.sub(r'[^\w\s]', '', new_sentence)